# Record Embedding

In [1]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

## Embedding using fastText
Details here: https://fasttext.cc/

In [2]:
import warnings
warnings.filterwarnings('ignore')
import fastText
from fastText import train_unsupervised
import gensim
from gensim.models import FastText

Convert each tuple into a row

In [3]:

#df = pd.read_csv('truth_values_1100_tuples.csv',dtype=object, encoding='utf8')
df = pd.read_csv("census_clean_unflatten.csv",dtype=object, index_col=False)
#df = df.drop(columns=['Height (inches)', 'Weight (lbs)'])
#df.rename(columns={'City.1': 'City'}, inplace=True)
df.replace(np.nan,'empty',inplace = True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(3)

,age,class_of_worker,education,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,race,hispanic_origin,sex,...,migration_code_move_within_reg,live_in_this_house_1_year_ago,migration_prev_res_in_sunbelt,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,year,income
0,7,not in universe,children,not in universe,never married,not in universe or children,not in universe,white,all other,male,...,nonmover,yes,not in universe,united-states,united-states,united-states,native- born in the united states,0,94,- 50000.
1,17,local government,10th grade,high school,never married,education,protective services,white,all other,female,...,nonmover,yes,not in universe,united-states,united-states,united-states,native- born in the united states,0,94,- 50000.
2,23,private,high school graduate,not in universe,never married,business and repair services,precision production craft & repair,white,all other,male,...,nonmover,yes,not in universe,united-states,united-states,united-states,native- born in the united states,0,94,- 50000.


In [4]:
#training the model with truth data.
import tempfile
data = []
dfList = df.values.tolist()
for line in dfList:
    line = [l.lower() for l in line]
    data.append(line)

with tempfile.NamedTemporaryFile(delete=False) as tmpf:
            for line in dfList:
                line = [l.lower() for l in line]
                tmpf.write("{}\n".format(line).encode())
            tmpf.flush()

In [5]:
#model_hosp = train_unsupervised(input=tmpf.name, model='skipgram', ws=df.shape[1])

In [ ]:
model_census = FastText(data, min_count=1, workers=8, iter=500, window=df.shape[1], sg=1, word_ngrams=1)

from numpy import dot
from numpy.linalg import norm

class FastTextNN:
    def __init__(self, ft_model, ft_matrix=None):
        self.ft_model = ft_model        
        self.ft_words = ft_model.get_words()
        #print(self.ft_words)
        self.word_frequencies = dict(zip(*ft_model.get_words(include_freq=True)))
        #print(self.word_frequencies)
        self.ft_matrix = ft_matrix
        if self.ft_matrix is None:
            self.ft_matrix = np.empty((len(self.ft_words), ft_model.get_dimension()))
            for i, word in enumerate(self.ft_words):
                self.ft_matrix[i,:] = ft_model.get_word_vector(word)
                
    def find_nearest_neighbor(self, query, vectors, n=10,  cossims=None):
        """
        query is a 1d numpy array corresponding to the vector to which you want to
        find the closest vector
        vectors is a 2d numpy array corresponding to the vectors you want to consider

        cossims is a 1d numpy array of size len(vectors), which can be passed for efficiency
        returns the index of the closest n matches to query within vectors and the cosine similarity (cosine the angle between the vectors)

        """
        at = []
        if cossims is None:
            minVal = 32
            #print(vectors.shape)
            cossims = np.matmul(vectors, query, out=cossims)
            for i in range(len(self.ft_words)):
                a = dot(query, self.ft_model.get_word_vector(self.ft_words[i]))/(norm(query)*norm(self.ft_model.get_word_vector(self.ft_words[i])))
                if a < range(minVal, minVal+10):
                    minVal = a
                    at.append(a, self.ft_words[i])
        
        norms = np.sqrt((query**2).sum() * (vectors**2).sum(axis=1))
        cossims = cossims/norms
        result_i = np.argpartition(-cossims, range(n+1))[1:n+1]
        return list(zip(result_i, cossims[result_i]))

    def nearest_words(self, word, n=15, word_freq=None):
        result = self.find_nearest_neighbor(self.ft_model.get_word_vector(word), self.ft_matrix, n=n)
        
        if word_freq:
            return [(self.ft_words[r[0]], r[1]) for r in result if self.word_frequencies[self.ft_words[r[0]]] >= word_freq]
        else:
            return [(self.ft_words[r[0]], r[1]) for r in result]
nn = FastTextNN(model_hosp)
nn.nearest_words('yxs', word_freq=15)

In [ ]:
model_census.save("CensusFastText.w2v")